In [ ]:
pip install opencv-python

In [ ]:
pip install opencv-python pyzbar numpy

In [ ]:
conda install -c conda-forge pyzbar zbar

In [1]:
import cv2
import os
from pyzbar.pyzbar import decode as pyzbar_decode

# Input and output directories
image_dir = "train_images"   # folder with your images
output_dir = "annotated_images"
os.makedirs(output_dir, exist_ok=True)

MIN_SIZE = 50  # minimum size (pixels) of detected QR box side

for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    h, w = img.shape[:2]
    boxes = []

    # --- Use Pyzbar ---
    decoded = pyzbar_decode(img)
    for obj in decoded:
        x, y, bw, bh = obj.rect
        if bw > MIN_SIZE and bh > MIN_SIZE:   # filter out tiny detections
            boxes.append([x, y, x + bw, y + bh])

    # --- Use OpenCV QRCodeDetector as backup ---
    qr_detector = cv2.QRCodeDetector()
    retval, decoded_info, points, _ = qr_detector.detectAndDecodeMulti(img)
    if points is not None:
        for p in points.astype(int):
            x_min, y_min = p[:, 0].min(), p[:, 1].min()
            x_max, y_max = p[:, 0].max(), p[:, 1].max()
            if (x_max - x_min) > MIN_SIZE and (y_max - y_min) > MIN_SIZE:
                boxes.append([x_min, y_min, x_max, y_max])

    # --- Draw final bounding boxes ---
    for (x1, y1, x2, y2) in boxes:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Save annotated image
    cv2.imwrite(os.path.join(output_dir, img_name), img)

print("✅ Annotated images saved in 'annotated_images'.")


✅ Annotated images saved in 'annotated_images'.


In [2]:
import cv2
import os
from pyzbar.pyzbar import decode as pyzbar_decode
from PIL import Image

# Input and output directories
image_dir = "train_images"   # folder with your images
output_dir = "datasets/qr"      # YOLO dataset folder

os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

MIN_SIZE = 50  # minimum size (pixels) of detected QR box side

def convert_bbox(size, box):
    """Convert [x_min, y_min, x_max, y_max] → YOLO format (x_center, y_center, w, h normalized)"""
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_min, y_min, x_max, y_max = box
    x = (x_min + x_max) / 2.0
    y = (y_min + y_max) / 2.0
    w = x_max - x_min
    h = y_max - y_min
    x *= dw
    y *= dh
    w *= dw
    h *= dh
    return (x, y, w, h)

for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    h, w = img.shape[:2]
    boxes = []

    # --- Use Pyzbar ---
    decoded = pyzbar_decode(img)
    for obj in decoded:
        x, y, bw, bh = obj.rect
        if bw > MIN_SIZE and bh > MIN_SIZE:
            boxes.append([x, y, x + bw, y + bh])

    # --- Use OpenCV QRCodeDetector as backup ---
    qr_detector = cv2.QRCodeDetector()
    retval, decoded_info, points, _ = qr_detector.detectAndDecodeMulti(img)
    if points is not None:
        for p in points.astype(int):
            x_min, y_min = p[:, 0].min(), p[:, 1].min()
            x_max, y_max = p[:, 0].max(), p[:, 1].max()
            if (x_max - x_min) > MIN_SIZE and (y_max - y_min) > MIN_SIZE:
                boxes.append([x_min, y_min, x_max, y_max])

    # --- Save YOLO labels ---
    if boxes:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        txt_path = os.path.join(output_dir, "labels", txt_name)

        with open(txt_path, "w") as f:
            for (x1, y1, x2, y2) in boxes:
                x, y, w_norm, h_norm = convert_bbox((w, h), [x1, y1, x2, y2])
                f.write(f"0 {x} {y} {w_norm} {h_norm}\n")  # "0" = class id for QR code

    # --- Copy images to dataset folder ---
    cv2.imwrite(os.path.join(output_dir, "images", img_name), img)

print("✅ YOLO dataset ready in datasets/qr/images + datasets/qr/labels")


✅ YOLO dataset ready in datasets/qr/images + datasets/qr/labels


In [3]:
import cv2
import os
import shutil
import random
from pyzbar.pyzbar import decode as pyzbar_decode
from PIL import Image

# Input raw images
image_dir = "train_images"   # your original images

# Output dirs
output_dir = "datasets/qr"
os.makedirs(output_dir, exist_ok=True)

for split in ["train", "val"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Parameters
MIN_SIZE = 50   # filter tiny detections
VAL_SPLIT = 0.2 # 20% validation

# Collect all images
all_images = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg','.png'))]
random.shuffle(all_images)
val_count = int(len(all_images) * VAL_SPLIT)
val_set = set(all_images[:val_count])

def convert_bbox(size, box):
    """Convert [x_min, y_min, x_max, y_max] → YOLO format (x_center, y_center, w, h normalized)"""
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_min, y_min, x_max, y_max = box
    x = (x_min + x_max) / 2.0
    y = (y_min + y_max) / 2.0
    w = x_max - x_min
    h = y_max - y_min
    x *= dw
    y *= dh
    w *= dw
    h *= dh
    return (x, y, w, h)

qr_detector = cv2.QRCodeDetector()

for img_name in all_images:
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    h, w = img.shape[:2]
    boxes = []

    # --- Pyzbar detection ---
    decoded = pyzbar_decode(img)
    for obj in decoded:
        x, y, bw, bh = obj.rect
        if bw > MIN_SIZE and bh > MIN_SIZE:
            boxes.append([x, y, x + bw, y + bh])

    # --- OpenCV QRCodeDetector as backup ---
    retval, decoded_info, points, _ = qr_detector.detectAndDecodeMulti(img)
    if points is not None:
        for p in points.astype(int):
            x_min, y_min = p[:, 0].min(), p[:, 1].min()
            x_max, y_max = p[:, 0].max(), p[:, 1].max()
            if (x_max - x_min) > MIN_SIZE and (y_max - y_min) > MIN_SIZE:
                boxes.append([x_min, y_min, x_max, y_max])

    # --- Save YOLO label file ---
    if boxes:
        yolo_lines = []
        for (x1, y1, x2, y2) in boxes:
            x, y, w_norm, h_norm = convert_bbox((w, h), [x1, y1, x2, y2])
            yolo_lines.append(f"0 {x} {y} {w_norm} {h_norm}\n")
            # draw box for debug
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

        split = "val" if img_name in val_set else "train"

        # save image
        cv2.imwrite(os.path.join(output_dir, "images", split, img_name), img)

        # save labels
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        txt_path = os.path.join(output_dir, "labels", split, txt_name)
        with open(txt_path, "w") as f:
            f.writelines(yolo_lines)

print("✅ YOLO dataset created with train/val split and debug images saved.")

# --- Create data.yaml ---
yaml_path = os.path.join(output_dir, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(
f"""train: {output_dir}/images/train
val: {output_dir}/images/val

nc: 1
names: ["qr"]
"""
)
print(f"✅ data.yaml created at {yaml_path}")


✅ YOLO dataset created with train/val split and debug images saved.
✅ data.yaml created at datasets/qr\data.yaml


In [4]:
pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/d5/9f/224adf14968bd313161fb029c34b12ae96fa7a670c85ff6d8289aaa46523/ultralytics-8.3.204-py3-none-any.whl.metadata
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/93/40/3415d890eb357b25a8e0a215d32365a88ecc75a283f75c4e919024b22d97/torchvision-0.23.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/6b/80/1791ac226bb989bef30fe8fde752b2021b6ec5dfd6e880262596aedf4c05/polars-1.34.0-py3-none-any.whl.metadata
  Obtaining dependency information for ultralytics-thop>=2.0.0 from https://files.pythonhosted.org/packages/b7/02/de5b1f012a9eb319fdd724049902d7b6886b694c281059e5d734296b74af/ultralytics_thop-2.0.17-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")


model.train(
    data="E:/QR_Dataset/QR_Dataset/src/datasets/qr/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    name="qr_detector_aug",
    hsv_h=0.02,
    hsv_s=0.8,
    hsv_v=0.5,
    degrees=15,
    translate=0.2,
    scale=0.6,
    shear=5.0,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.3
)


Ultralytics 8.3.204  Python-3.11.4 torch-2.8.0+cpu CPU (Intel Core i3-1005G1 1.20GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:/QR_Dataset/QR_Dataset/src/datasets/qr/data.yaml, degrees=15, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.3, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=qr_detector_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 8.1s6.9ss
                   all         40        195      0.998      0.759      0.778       0.51

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       7/50         0G     0.8174      0.724     0.8558        105        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:203.3sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.7s6.5ss
                   all         40        195          1      0.758      0.777       0.53

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       8/50         0G     0.7565     0.6667     0.8396         64        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:193.3sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 8.2s6.8ss
                   all        

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.3s6.2ss
                   all         40        195      0.993      0.767      0.767      0.683

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      27/50         0G     0.5472     0.4153     0.8027         72        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:193.4sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.3s6.2ss
                   all         40        195      0.999      0.759      0.769      0.715

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      28/50         0G     0.5175     0.4031     0.8073         38        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:203.3sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.3s6.1ss
                   all        

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.4s6.3ss
                   all         40        195      0.999      0.769       0.77      0.719

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      46/50         0G     0.3591     0.3267     0.7774         25        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:153.1sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.4s6.2ss
                   all         40        195          1      0.764       0.77       0.72

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      47/50         0G     0.3634     0.3402     0.7827         18        640: 100% ━━━━━━━━━━━━ 20/20 0.3it/s 1:163.2sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.4it/s 7.9s6.7ss
                   all        

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015614DB8950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [3]:
model.val()

Ultralytics 8.3.204  Python-3.11.4 torch-2.8.0+cpu CPU (Intel Core i3-1005G1 1.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 1218.8231.3 MB/s, size: 1859.1 KB)
val: Scanning E:\QR_Dataset\QR_Dataset\src\datasets\qr\labels\val.cache... 40 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 40/40  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 0.7it/s 7.4s2.1ss
                   all         40        195      0.993      0.768      0.767      0.726
Speed: 1.3ms preprocess, 96.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to C:\Users\DELL\runs\detect\qr_detector_aug2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015617FF7690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [9]:
from ultralytics import YOLO

# Load the best weights from run 5
model = YOLO(r"C:\Users\DELL\Dropbox\My PC (DELLINSPIRON3935)\Downloads\best.pt")

# Run predictions on your 50 reserved test images
results = model.predict(
    source=r"E:\QR_Dataset\QR_Dataset\test_images",  # replace with your test image folder path
    conf=0.25,
    save=True,
    save_txt=True
)



image 1/50 E:\QR_Dataset\QR_Dataset\test_images\img201.jpg: 640x384 1 qr, 128.1ms
image 2/50 E:\QR_Dataset\QR_Dataset\test_images\img202.jpg: 384x640 1 qr, 86.1ms
image 3/50 E:\QR_Dataset\QR_Dataset\test_images\img203.jpg: 640x384 6 qrs, 78.4ms
image 4/50 E:\QR_Dataset\QR_Dataset\test_images\img204.jpg: 640x384 5 qrs, 96.9ms
image 5/50 E:\QR_Dataset\QR_Dataset\test_images\img205.jpg: 640x384 5 qrs, 97.1ms
image 6/50 E:\QR_Dataset\QR_Dataset\test_images\img206.jpg: 640x384 4 qrs, 92.6ms
image 7/50 E:\QR_Dataset\QR_Dataset\test_images\img207.jpg: 640x384 5 qrs, 114.1ms
image 8/50 E:\QR_Dataset\QR_Dataset\test_images\img208.jpg: 640x384 4 qrs, 146.0ms
image 9/50 E:\QR_Dataset\QR_Dataset\test_images\img209.jpg: 640x384 4 qrs, 80.7ms
image 10/50 E:\QR_Dataset\QR_Dataset\test_images\img210.jpg: 640x384 7 qrs, 77.4ms
image 11/50 E:\QR_Dataset\QR_Dataset\test_images\img211.jpg: 640x384 1 qr, 82.4ms
image 12/50 E:\QR_Dataset\QR_Dataset\test_images\img212.jpg: 640x384 2 qrs, 78.2ms
image 13/50 